In [ ]:
import pandas as pd

In [2]:
# TODO:
# most common words by different categories
# lda topic distribution by different categories
# better filtering: exclude text with '(number)' or '(LETTER)' or 'as follows:'

# DONE:
# duplicate speech elemination, report its count
# some speeches are like law text submitted the following e.g. df.contents.iloc[845]
# related with above: delete extra spaces in speech (probably they occur in pure law texts)
# text token len in average and per speaker, and per party
# better filtering: exclude too long texts from via max token_count 

### Preprocessing

In [ ]:
train_df = pd.read_csv('/cluster/scratch/goezsoy/nlp_lss_datasets/processed_df_train.csv')

In [3]:
df = pd.read_pickle('tokenized-records.pkl')

df = df[['bioguide_id','birthday','gender','term_type','term_start','term_end','term_state','term_party','speech']]

# preprocessing: eleminate duplicate speeches, and remove '/n'
unique_speeches_nested = df.groupby('bioguide_id')['speech'].agg(lambda x: list(set(x.values)))

unique_speeches_single_list = []
for idx,row in unique_speeches_nested.reset_index(drop=False).iterrows():
    for temp_speech in row.speech:
        temp_speech = temp_speech.replace('\n','')
        unique_speeches_single_list.append((row.bioguide_id,temp_speech))

unique_speeches_single_df = pd.DataFrame(unique_speeches_single_list,columns=['bioguide_id','speech'])

# row[0]!='.' speeches are not speech but text snippets describing the law 
processed_df = unique_speeches_single_df[unique_speeches_single_df.speech.apply(lambda row: row[0]=='.')]

# add speaker metadata to processed speeches using pd.merge
df = df.drop('speech',axis=1)
df = df.drop_duplicates(subset='bioguide_id')
processed_df= processed_df.merge(df,on='bioguide_id',how='inner')

### Analysis

In [4]:
processed_df['token_count'] = processed_df['speech'].apply(lambda row: len(row.split()))

In [5]:
# word count statistics by political parties
processed_df.groupby('term_party')['token_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,95%,99%,max
term_party,,,,,,,,,,
Democrat,110881.0,466.130924,3056.491393,2.0,37.0,167.0,477.0,1639.00,3588.00,364497.0
Independent,1610.0,664.330435,1396.209942,5.0,37.0,189.5,817.5,2573.65,4496.35,23765.0
Republican,95731.0,441.777105,2327.703812,2.0,35.0,160.0,432.0,1672.50,3687.70,373631.0


In [6]:
# word count statistics by gender
processed_df.groupby('gender')['token_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,95%,99%,max
gender,,,,,,,,,,
F,37039.0,465.252896,2907.392255,2.0,45.0,189.0,497.0,1506.1,3210.48,286216.0
M,171183.0,454.565564,2696.512727,2.0,35.0,158.0,447.0,1697.9,3730.00,373631.0


In [7]:
# word count statistics by Senate or House
processed_df.groupby('term_type')['token_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

,count,mean,std,min,25%,50%,75%,95%,99%,max
term_type,,,,,,,,,,
H,123861.0,380.770606,3224.614615,2.0,40.0,166.0,365.0,986.0,2953.4,373631.0
S,84361.0,567.605540,1782.751186,2.0,32.0,156.0,711.0,2236.0,4126.4,328056.0


In [8]:
# word count statistics by state sorted with descending record size
processed_df.groupby('term_state')['token_count'].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99]).sort_values(by='count',ascending=False)

,count,mean,std,min,25%,50%,75%,95%,99%,max
term_state,,,,,,,,,,
CA,19853.0,397.334055,2169.364070,3.0,40.00,163.0,402.00,1162.40,3565.08,186901.0
TX,13241.0,556.848501,2993.248709,4.0,40.00,178.0,498.00,2055.00,5149.00,217282.0
NY,9761.0,505.963528,5762.826539,5.0,32.00,162.0,426.00,1294.00,2671.00,328056.0
KY,8313.0,308.711175,1552.368602,3.0,26.00,69.0,303.00,1191.20,2470.68,119516.0
IL,7764.0,495.067491,1088.918112,5.0,37.00,168.0,524.00,2139.00,4021.00,38728.0
NV,7747.0,243.233639,1074.516408,4.0,28.00,65.0,229.00,985.10,1929.86,63625.0
MI,7573.0,394.971874,2591.983667,4.0,27.00,118.0,356.00,1190.20,2911.36,146282.0
FL,6559.0,345.181430,857.064557,2.0,52.00,174.0,391.00,1139.10,2722.92,41511.0
MD,6404.0,502.716271,4719.900858,2.0,43.00,194.0,533.00,1600.00,3277.97,364497.0


In [9]:
# keep all until 99th percentile (removing outliers)
processed_df = processed_df[processed_df['token_count'] <= 3646.790000]
pd.to_pickle(processed_df,'processed_df.pkl')

In [16]:
processed_df.bioguide_id.value_counts()

R000146    6438
M000355    6437
D000563    3543
M000312    2151
L000174    2049
           ... 
K000180       1
C001092       1
C001089       1
C001066       1
U000038       1
Name: bioguide_id, Length: 882, dtype: int64

In [17]:
import spacy
nlp = spacy.load('en_core_web_sm')

from gensim.corpora import Dictionary

dfs = processed_df[processed_df.bioguide_id=='L000174']
dfs['cleaned_lda'] = dfs['speech'].apply(lambda row: [t.lower_ for t in nlp(row) if not t.is_punct and not t.is_stop])
dictionary = Dictionary(dfs['cleaned_lda'])
doc_term_matrix = [dictionary.doc2bow(doc) for doc in dfs['cleaned_lda']]

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from gensim.models.ldamodel import LdaModel
lda = LdaModel(doc_term_matrix, num_topics=10, id2word = dictionary, passes=3)
pyLDAvis.gensim.prepare(lda, doc_term_matrix, dictionary)

/var/folders/r_/jwmzgr6s6_762bz0h7x_qll40000gn/T/ipykernel_11405/1844123722.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['cleaned_lda'] = dfs['speech'].apply(lambda row: [t.lower_ for t in nlp(row) if not t.is_punct and not t.is_stop])
/Users/gokberk/miniconda3/envs/nlp_lss/lib/python3.8/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/Users/gokberk/miniconda3/envs/nlp_lss/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp im

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.054917  0.118859       1        1  21.287361
4      0.051633  0.008063       2        1  15.032216
1     -0.149536  0.138430       3        1  14.073074
7     -0.162243 -0.184612       4        1  13.120163
0     -0.041045 -0.001731       5        1  10.782072
8      0.135611 -0.084096       6        1   8.966821
9     -0.197730 -0.006199       7        1   7.416842
3      0.073278 -0.042665       8        1   5.380021
6      0.155841 -0.094543       9        1   3.316685
5      0.079276  0.148494      10        1   0.624745, topic_info=           Term          Freq         Total Category  logprob  loglift
206              20117.000000  20117.000000  Default  30.0000  30.0000
0                14179.000000  14179.000000  Default  29.0000  29.0000
207               4621.000000   4621.000000  Default  28.0000  28.0000
33            `   9480.000000   9480.000000  Default  27.0000  27.0000
1609   nominees   2751.000000   2751.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
963     program      7.342049    969.203815  Topic10  -6.5000   0.1927
206                  8.071383  20117.639625  Topic10  -6.4053  -2.7454
1066    vermont      7.240465   1803.707003  Topic10  -6.5140  -0.4423
584     support      7.041157   2131.348100  Topic10  -6.5419  -0.6372
61    companies      6.859281    241.613904  Topic10  -6.5681   1.5138

[839 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.180758       
0         2  0.212636       
0         3  0.042527       
0         4  0.119330       
0         5  0.122010       
...     ...       ...    ...
19        6  0.291160  yield
19        7  0.003386  yield
19        8  0.003386  yield
19        9  0.538308  yield
19       10  0.003386  yield

[3101 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 8, 1, 9, 10, 4, 7, 6])